In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'epa', 'local', 'met', 'petsitly_marketing', 'realestate-Project-3', 'travel_db', 'uk_food']


In [4]:
#database to a variable name
db = mongo['realestate-Project-3']

In [5]:
# review the collections in our database
print(db.list_collection_names())

['All-Listings']


In [6]:
# assign the collection to a variable
all_listings = db['All-Listings']

In [7]:
pprint(all_listings.find_one())

{'Accuracy Score': 0.7,
 'Accuracy Type': 'street_center',
 'Address 1': '0 Evergreen Dr. Lot 2',
 'City': 'Navasota',
 'Country': 'US',
 'County': 'Grimes County',
 'Date Inspected': datetime.datetime(2022, 2, 22, 0, 0),
 'Full Address': 'Address:\n'
                 '      \n'
                 '        0 Evergreen Dr. Lot 2\n'
                 '              \n'
                 '        Locality:\n'
                 '    \n'
                 '                      77868 TX\n'
                 '            United States',
 'Latitude': 30.282444,
 'Longitude': -95.921109,
 'Property ID': 205090,
 'Property Shown': '✔',
 'Property Type': 'Land',
 'Sale Date': 'Tuesday, February 22, 2022',
 'Source': 'TIGER/Line® dataset from the US Census Bureau',
 'State': 'TX',
 'Status': 'Available',
 'Street': 'Evergreen Forest Ln',
 'Subdivision': 'BLUEBONNET CRTY',
 'Unit Number': 2,
 'Unit Type': 'Lot',
 'ZIP': 77868,
 'Zip': 77868,
 '_id': ObjectId('65162f1261c80aafabd077a4')}


In [8]:
#Total Accounts by city
query = {'City': 'Houston'}

# Print the number of results
print("Number of documents in result:", all_listings.count_documents(query))

Number of documents in result: 312


In [ ]:
#Show the Available and Sold homes by city

In [9]:
pipeline = [
    {
        '$match': {
            'Status': {'$in': ['Available', 'Sold']}
        }
    },
    {
        '$group': {
            '_id': '$City',
            'totalAvailable': {
                '$sum': {
                    '$cond': [
                        {'$eq': ['$Status', 'Available']},
                        1,
                        0
                    ]
                }
            },
            'totalSold': {
                '$sum': {
                    '$cond': [
                        {'$eq': ['$Status', 'Sold']},
                        1,
                        0
                    ]
                }
            }
        }
    },
    {
        '$project': {
            '_id': 1,
            'totalAvailable': 1,
            'totalSold': 1
        }
    },
    {
        '$sort': {
            '_id': 1  # Sort by '_id' (city name) in ascending order
        }
    }
]

result = list(all_listings.aggregate(pipeline))

for item in result:
    print(f"City: {item['_id']}, Total Available Properties: {item['totalAvailable']}, Total Sold Properties: {item['totalSold']}")

City: Albany, Total Available Properties: 1, Total Sold Properties: 0
City: Alice, Total Available Properties: 2, Total Sold Properties: 0
City: Alto, Total Available Properties: 1, Total Sold Properties: 0
City: Alvarado, Total Available Properties: 1, Total Sold Properties: 0
City: Alvin, Total Available Properties: 1, Total Sold Properties: 0
City: Angleton, Total Available Properties: 13, Total Sold Properties: 0
City: Anson, Total Available Properties: 2, Total Sold Properties: 0
City: Aransas Pass, Total Available Properties: 1, Total Sold Properties: 0
City: Archer City, Total Available Properties: 1, Total Sold Properties: 0
City: Arp, Total Available Properties: 1, Total Sold Properties: 0
City: Athens, Total Available Properties: 3, Total Sold Properties: 1
City: Atlanta, Total Available Properties: 1, Total Sold Properties: 0
City: Austell, Total Available Properties: 1, Total Sold Properties: 0
City: Austin, Total Available Properties: 1, Total Sold Properties: 0
City: Avin

In [ ]:
# Search by city

In [12]:
# Import necessary libraries
import pandas as pd

# Prompt the user for the city and availability status
city_name = input("What city are you looking for? ")
availability_status = input("Are you looking for 'Sold' or 'Available' homes? ")


# Validate the availability status input
if availability_status not in ['sold', 'available']:
    print("Invalid availability status. Please enter 'sold' or 'available'.")
else:
    # Query the database based on user input
    property_query = {
        'City': city_name,
        'Status': availability_status.capitalize()
    }

    # Find properties that match the query
    matching_properties = all_listings.find(property_query)
    

    # Count the number of matching documents
    matching_properties_count = all_listings.count_documents({})

    # Check if there are matching properties
    if matching_properties_count > 0:
        # Create a DataFrame to display the results
        result_df = pd.DataFrame(matching_properties, columns=['Property ID', 'Street'])
        
        # Display the DataFrame
        print(result_df)
    else:
        pprint(f"No {availability_status} properties found in {city_name}.")

What city are you looking for? Beaumont
Are you looking for 'Sold' or 'Available' homes? sold
   Property ID       Street
0       205083          NaN
1       201066   Irving Ave
2       198020     Ollie St
3       205081  Highland Dr
4       205017  Wildwood Dr
5       191003       Hwy 90
6       198014      Tram Rd


In [31]:
selected_property_id = input("Enter the Property ID to view details: ")

# Find the property by Property ID
selected_property = all_listings.find_one({'Property ID': int(selected_property_id)})

# Check if the property was found
if selected_property:
    # Display the details of the selected property
    print("\nDetails of Selected Property:")
    for key, value in selected_property.items():
        print(f"{key}: {value}")
else:
    print(f"Property with ID {selected_property_id} not found.")

Enter the Property ID to view details: 205017

Details of Selected Property:
_id: 65162f1261c80aafabd07d2b
Property ID: 205017
Property Type: Residential
Full Address: Address:
      
        5145 Wildwood Dr
              
        Locality:
    
                     77708 TX
            United States
Address 1: 5145 Wildwood Dr
City: Beaumont
ZIP: 77708
State: TX
Status: Sold
Specialist: Orion Rodriguez
Sale Date: Thursday, June 01, 2023
Price: 222,999
Down 1: 9,999
Better price: 221,500
Down 2: 10,999
Best price: 220,500
Down 3: 12999.0
Beds: 4
Baths: 2
Sq: {'Ft': '1,894'}
Garage: 2.0
Year Built: 1978
Property Shown: ✔
County: Jefferson County
Subdivision: WOODLAND ESTATES
Date Inspected: 2023-05-17 00:00:00
Latitude: 30.138068
Longitude: -94.160619
Accuracy Score: 1
Accuracy Type: rooftop
Number: 5145
Street: Wildwood Dr
Zip: 77708
Country: US
Source: City of Beaumont
